# Datathon

## Pablo Lázaro Herrasti y Manuel Jesús Galán Moreu

### Imports

In [157]:
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
import warnings
warnings.filterwarnings("ignore")
import xgboost as xgb


### SKLEARN

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler ### import Standard and MinMax Preprocessing 
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC ### Support Vector Machine
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier ### Random forest and XGBoosting Classifier
from xgboost import XGBClassifier
from sklearn.feature_selection import RFE

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from sklearn import datasets

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score, train_test_split

from mlxtend.plotting import plot_learning_curves
from mlxtend.plotting import plot_decision_regions

### Reading data

In [158]:
dir_data = 'C:/Users/manuel.jesus.galan.1/Desktop/Datathon/Datasets/'
elephant_dir = 'elephant.csv'
ring_dir = 'ring.csv'
yeast1_dir = 'yeast1.csv'

In [159]:
df_elephant = pd.read_csv(dir_data + elephant_dir, sep=';', skipinitialspace=True)
df_ring = pd.read_csv(dir_data + ring_dir, sep=';', skipinitialspace=True)
df_yeast1 = pd.read_csv(dir_data + yeast1_dir, sep=';', skipinitialspace=True)

### Showing data

In [160]:
print('Number of rows: ' + str(len(df_elephant)))
print('Number of columns: ' + str(len(df_elephant.columns)))
df_elephant.head()

Number of rows: 1391
Number of columns: 231


,Atr-1,Atr-2,Atr-3,Atr-4,Atr-5,Atr-6,Atr-7,Atr-8,Atr-9,Atr-10,...,Atr-222,Atr-223,Atr-224,Atr-225,Atr-226,Atr-227,Atr-228,Atr-229,Atr-230,CLASS
0,"-0,28698","-0,624297","-0,679333","-0,455715","0,371213","-0,415471","-0,549112","-0,4722","0,363383","0,33419",...,0,"-0,049855",120.049,"-0,078862","-0,021452",0,0,"-0,014952","-0,021097",1
1,"0,363763","0,146879","-0,83422","0,819986","0,504474",167.231,"-0,774634","0,76555","0,355781","0,215197",...,0,"-0,049855",129.994,"-0,078862","-0,021452",0,0,"-0,014952","-0,021097",1
2,"-0,15592","-0,139025",-115.844,246.346,"-0,463596",-138.827,"-0,945256",260.892,"-0,049767","0,846782",...,0,"-0,049855","0,031608","0,182763","-0,021452",0,0,"-0,014952","-0,021097",1
3,-122.972,-119.093,"-0,364069",-132.571,"-0,005645","-0,740226","-0,468026",-131.328,"-0,082819","0,857917",...,0,"-0,049855","0,591019","-0,078862","-0,021452",0,0,"-0,014952","-0,021097",1
4,"0,9162","0,265188","-0,252377",256.115,"-0,128087","-0,105623","-0,302654",259.287,"-0,167432","-0,626084",...,0,"-0,049855","-0,114025","0,167061","-0,021452",0,0,"-0,014952","-0,021097",1


In [161]:
print('Number of rows: ' + str(len(df_ring)))
print('Number of columns: ' + str(len(df_ring.columns)))

df_ring.head()

Number of rows: 7400
Number of columns: 21


,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A12,A13,A14,A15,A16,A17,A18,A19,A20,Class
0,849,2177,598,1689,3114,-3406,3799,-2642,1578,-181,...,1633,4664,1081,-1172,-166,578,1664,309,-3028,0
1,947,543,782,-449,-8,1316,756,-61,1604,1686,...,-382,1259,608,-2292,1008,2687,-1422,-606,2374,1
2,759,-314,898,-1743,2003,1117,911,136,-489,-144,...,112,598,474,2026,1431,-336,79,1123,302,1
3,531,1374,517,1226,1122,-4,-1227,2277,1083,888,...,1063,2454,587,-744,1216,916,-476,6,-477,1
4,-1443,1065,2071,207,-865,-786,2655,-366,971,-341,...,4195,-117,-2694,14,4097,1356,-944,-602,2348,0


In [162]:
print('Number of rows: ' + str(len(df_yeast1)))
print('Number of columns: ' + str(len(df_yeast1.columns)))
df_yeast1.head()

Number of rows: 1484
Number of columns: 9


,Mcg,Gvh,Alm,Mit,Erl,Pox,Vac,Nuc,Class
0,"0,58","0,61","0,47","0,13","0,5",0,"0,48","0,22",negative
1,"0,43","0,67","0,48","0,27","0,5",0,"0,53","0,22",negative
2,"0,64","0,62","0,49","0,15","0,5",0,"0,53","0,22",negative
3,"0,58","0,44","0,57","0,13","0,5",0,"0,54","0,22",positive
4,"0,42","0,44","0,48","0,54","0,5",0,"0,48","0,22",negative


### First cleaning

#### Funciones útiles

In [163]:
def obtain_labels(df):
    
    if len(df.columns) == 231 or len(df.columns) == 21:
        return list(df.iloc[:,-1])
    else:
        return [0 if label == 'negative' else 1 for label in list(df.iloc[:,-1])]

In [164]:
def preprocess_dataframe(df, mode):
    
    ### Erase the duplicate samples
    df = df.drop_duplicates() 
    
    ### Obtain labels if we are in training
    if mode == 'Train':
        labels = obtain_labels(df)
    else: 
        labels = []
    
    ### Clean columns that are uniques and does not give us any information
    eliminate_columns = []
    df = df.iloc[:,:-1]
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = pd.to_numeric(df[column].str.replace(',','.'), errors='coerce')
        if len(df[column].unique()) == 1:
            eliminate_columns.append(column)
    df_drop = df.drop(eliminate_columns, axis=1)

    return df_drop, labels, eliminate_columns

In [165]:
data_elephant, labels_elephant, eliminate_elephant = preprocess_dataframe(df_elephant, 'Train')
data_ring, labels_ring, eliminate_ring = preprocess_dataframe(df_ring, 'Train')
data_yeast1, labels_yeast1, eliminate_yeast1 = preprocess_dataframe(df_yeast1,'Train')

### Showing data preprocessed

In [166]:
print('Number of columns: ' + str(len(data_elephant.columns)))
print('Number of rows: ' + str(len(data_elephant)))
data_elephant.head()

Number of columns: 110
Number of rows: 1391


,Atr-1,Atr-2,Atr-3,Atr-4,Atr-5,Atr-6,Atr-7,Atr-8,Atr-9,Atr-10,...,Atr-211,Atr-215,Atr-216,Atr-217,Atr-218,Atr-219,Atr-223,Atr-224,Atr-225,Atr-226
0,-0.286980,-0.624297,-0.679333,-0.455715,0.371213,-0.415471,-0.549112,-0.47220,0.363383,0.334190,...,-0.037709,-0.242745,-0.173706,-0.308654,165.218000,-0.017565,-0.049855,120.049000,-0.078862,-0.021452
1,0.363763,0.146879,-0.834220,0.819986,0.504474,167.231000,-0.774634,0.76555,0.355781,0.215197,...,-0.037709,-0.265739,-0.149831,-0.321915,0.817139,-0.017565,-0.049855,129.994000,-0.078862,-0.021452
2,-0.155920,-0.139025,-115.844000,246.346000,-0.463596,-138.827000,-0.945256,260.89200,-0.049767,0.846782,...,-0.037709,-0.265739,0.502075,-0.399635,-0.102856,-0.017565,-0.049855,0.031608,0.182763,-0.021452
3,-122.972000,-119.093000,-0.364069,-132.571000,-0.005645,-0.740226,-0.468026,-131.32800,-0.082819,0.857917,...,-0.037709,-0.265739,-0.337459,-0.311496,11.019000,-0.017565,-0.049855,0.591019,-0.078862,-0.021452
4,0.916200,0.265188,-0.252377,256.115000,-0.128087,-0.105623,-0.302654,259.28700,-0.167432,-0.626084,...,-0.037709,-0.265739,133.835000,-0.378855,-0.102856,-0.017565,-0.049855,-0.114025,0.167061,-0.021452


In [167]:
print('Number of columns: ' + str(len(data_ring.columns)))
print('Number of rows: ' + str(len(data_ring)))
data_ring.head()

Number of columns: 20
Number of rows: 7400


,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20
0,849,2177,598,1689,3114,-3406,3799,-2642,1578,-181,-212,1633,4664,1081,-1172,-166,578,1664,309,-3028
1,947,543,782,-449,-8,1316,756,-61,1604,1686,264,-382,1259,608,-2292,1008,2687,-1422,-606,2374
2,759,-314,898,-1743,2003,1117,911,136,-489,-144,982,112,598,474,2026,1431,-336,79,1123,302
3,531,1374,517,1226,1122,-4,-1227,2277,1083,888,-111,1063,2454,587,-744,1216,916,-476,6,-477
4,-1443,1065,2071,207,-865,-786,2655,-366,971,-341,221,4195,-117,-2694,14,4097,1356,-944,-602,2348


In [168]:
print('Number of columns: ' + str(len(data_yeast1.columns)))
print('Number of rows: ' + str(len(data_yeast1)))
data_yeast1.head()

Number of columns: 8
Number of rows: 1453


,Mcg,Gvh,Alm,Mit,Erl,Pox,Vac,Nuc
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22
3,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22
4,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22


### Correlation data

Función que crea un diccionario con key: atributo y value: atributos más parecidos y su porcentaje de correlación en una tupla

In [169]:
def calculate_corr(df):
    return df.corr() 

In [170]:
def topn_corr(df, top_n):
    
    top_n = top_n+1
    correlation = calculate_corr(df)
    all_corr = {}
    for column in correlation.columns:
        top = correlation[column].nlargest(top_n)
        indexes = list((top).index)[1:]
        all_corr[column] = (indexes, list(top)[1:])
    
    return all_corr

In [171]:
corr_elephant = calculate_corr(data_elephant)
corr_ring = calculate_corr(data_ring)
corr_yeast1 = calculate_corr(data_yeast1)

In [172]:
def eliminate_corr_features(df, corr, threshold):
    columns = np.full((corr.shape[0],), True, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i+1, corr.shape[0]):
            if corr.iloc[i,j] >= threshold:
                if columns[j]:
                    columns[j] = False
    selected_columns = df.columns[columns]
    df = df[selected_columns]
    
    return df

In [173]:
data_elephant_nocorr = eliminate_corr_features(data_elephant, corr_elephant, 0.8)
data_ring_nocorr = eliminate_corr_features(data_ring, corr_ring, 0.8)
data_yeast1_nocorr = eliminate_corr_features(data_yeast1, corr_yeast1, 0.8)

In [174]:
print('Number of columns: ' + str(len(data_elephant_nocorr.columns)))
print('Number of rows: ' + str(len(data_elephant_nocorr)))

data_elephant_nocorr.head()

Number of columns: 106
Number of rows: 1391


,Atr-1,Atr-3,Atr-4,Atr-5,Atr-6,Atr-9,Atr-10,Atr-11,Atr-12,Atr-13,...,Atr-211,Atr-215,Atr-216,Atr-217,Atr-218,Atr-219,Atr-223,Atr-224,Atr-225,Atr-226
0,-0.286980,-0.679333,-0.455715,0.371213,-0.415471,0.363383,0.334190,154.853000,267.959000,-0.054535,...,-0.037709,-0.242745,-0.173706,-0.308654,165.218000,-0.017565,-0.049855,120.049000,-0.078862,-0.021452
1,0.363763,-0.834220,0.819986,0.504474,167.231000,0.355781,0.215197,12.667000,127.461000,-0.054535,...,-0.037709,-0.265739,-0.149831,-0.321915,0.817139,-0.017565,-0.049855,129.994000,-0.078862,-0.021452
2,-0.155920,-115.844000,246.346000,-0.463596,-138.827000,-0.049767,0.846782,124.899000,-0.736989,-0.054535,...,-0.037709,-0.265739,0.502075,-0.399635,-0.102856,-0.017565,-0.049855,0.031608,0.182763,-0.021452
3,-122.972000,-0.364069,-132.571000,-0.005645,-0.740226,-0.082819,0.857917,0.649394,34.339000,-0.054535,...,-0.037709,-0.265739,-0.337459,-0.311496,11.019000,-0.017565,-0.049855,0.591019,-0.078862,-0.021452
4,0.916200,-0.252377,256.115000,-0.128087,-0.105623,-0.167432,-0.626084,156.991000,0.629992,-0.054535,...,-0.037709,-0.265739,133.835000,-0.378855,-0.102856,-0.017565,-0.049855,-0.114025,0.167061,-0.021452


In [175]:
print('Number of columns: ' + str(len(data_ring_nocorr.columns)))
print('Number of rows: ' + str(len(data_ring_nocorr)))

data_ring_nocorr.head()

Number of columns: 20
Number of rows: 7400


,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20
0,849,2177,598,1689,3114,-3406,3799,-2642,1578,-181,-212,1633,4664,1081,-1172,-166,578,1664,309,-3028
1,947,543,782,-449,-8,1316,756,-61,1604,1686,264,-382,1259,608,-2292,1008,2687,-1422,-606,2374
2,759,-314,898,-1743,2003,1117,911,136,-489,-144,982,112,598,474,2026,1431,-336,79,1123,302
3,531,1374,517,1226,1122,-4,-1227,2277,1083,888,-111,1063,2454,587,-744,1216,916,-476,6,-477
4,-1443,1065,2071,207,-865,-786,2655,-366,971,-341,221,4195,-117,-2694,14,4097,1356,-944,-602,2348


In [176]:
print('Number of columns: ' + str(len(data_yeast1_nocorr.columns)))
print('Number of rows: ' + str(len(data_yeast1_nocorr)))

data_yeast1_nocorr.head()

Number of columns: 8
Number of rows: 1453


,Mcg,Gvh,Alm,Mit,Erl,Pox,Vac,Nuc
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22
3,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22
4,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22


### Data split

In [177]:
X1 = data_elephant_nocorr
X2 = data_ring_nocorr
X3 = data_yeast1_nocorr
Y1 = labels_elephant
Y2 = labels_ring
Y3 = labels_yeast1

In [178]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1,Y1,test_size=0.2, random_state=30, stratify=Y1)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2,Y2,test_size=0.2, random_state=30, stratify=Y2)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3,Y3,test_size=0.2, random_state=30, stratify=Y3)

### Support Vector Machine

In [26]:
steps = [('scaler', StandardScaler()), ('SVM', SVC())]
pipeline = Pipeline(steps)

In [27]:
parameters = {'SVM__C':[0.001,0.1,10,100,10e5], 'SVM__gamma':[0.1,0.01]}
grid = GridSearchCV(pipeline, param_grid=parameters, cv=5)

In [28]:
grid.fit(X_train1, y_train1)
print("Elephant accuracy = %3.4f" %(grid.score(X_test1,y_test1)))
fpr, tpr, thresholds = metrics.roc_curve(y_test1, grid.predict(X_test1))
print("Elephant AUC = %3.4f" %(metrics.auc(fpr, tpr)))
print(grid.best_params_)

KeyboardInterrupt: 

In [49]:
grid.fit(X_train2, y_train2)
print("Ring accuracy = %3.4f" %(grid.score(X_test2,y_test2)))
fpr, tpr, thresholds = metrics.roc_curve(y_test2, grid.predict(X_test2))
print("Ring AUC = %3.4f" %(metrics.auc(fpr, tpr)))
print(grid.best_params_)

Ring accuracy = 0.9811
Ring AUC = 0.9811
{'SVM__C': 0.1, 'SVM__gamma': 0.1}


In [50]:
grid.fit(X_train3, y_train3)
print("Yeast accuracy = %3.4f" %(grid.score(X_test3,y_test3)))
fpr, tpr, thresholds = metrics.roc_curve(y_test3, grid.predict(X_test3))
print("Yeast AUC = %3.4f" %(metrics.auc(fpr, tpr)))
print(grid.best_params_)

Yeast accuracy = 0.8179
Yeast AUC = 0.7055
{'SVM__C': 100, 'SVM__gamma': 0.01}


## Random forest

##### Grid Search 

In [179]:
steps = [('scaler', StandardScaler()), ('clf', RandomForestClassifier())]
pipeline = Pipeline(steps)

In [180]:
parameters = {'clf__n_estimators':[100, 300, 500, 800, 1200],
'clf__max_depth':[5, 8, 15, 25, 30],
'clf__min_samples_split':[2, 5, 10, 15, 100],
'clf__min_samples_leaf':[1, 2, 5, 10] 
}

In [181]:
grid = GridSearchCV(pipeline, param_grid=parameters, cv=5)

#### ELEPHANTS

##### Recursive Feature Elimination (RFE)

In [ ]:
rfe = RFE(grid, 4)
#start = time.process_time()
RFE_X_Train = rfe.fit_transform(X_train1,y_train1)
RFE_X_Test = rfe.transform(X_test1)

rfe = rfe.fit(RFE_X_Train,y_train1)

#print(time.process_time() - start)

print("Overall Accuracy using RFE: ", rfe.score(RFE_X_Test,y_test1))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test1, grid.predict(RFE_X_Test))
print("Elephant AUC = %3.3f" %(metrics.auc(fpr, tpr)))
print(grid.best_params_)

##### Normal

In [39]:
grid.fit(X_train1, y_train1)
print("Elephant accuracy = %3.3f" %(grid.score(X_test1,y_test1)))
fpr, tpr, thresholds = metrics.roc_curve(y_test1, grid.predict(X_test1))
print("Elephant AUC = %3.3f" %(metrics.auc(fpr, tpr)))
print(grid.best_params_)

KeyboardInterrupt: 

### RINGS

##### Recursive Feature Elimination (RFE)

In [ ]:
rfe = RFE(grid, 4)
#start = time.process_time()
RFE_X_Train = rfe.fit_transform(X_train2,y_train2)
RFE_X_Test = rfe.transform(X_test2)

rfe = rfe.fit(RFE_X_Train,y_train2)

#print(time.process_time() - start)

print("Overall Accuracy using RFE: ", rfe.score(RFE_X_Test,y_test2))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test2, grid.predict(RFE_X_Test))
print("Elephant AUC = %3.3f" %(metrics.auc(fpr, tpr)))
print(grid.best_params_)

##### Normal

In [ ]:
clf.fit(X_train2,y_train2)
print("Ring accuracy = %3.3f" %(grid.score(X_test2,y_test2)))
fpr, tpr, thresholds = metrics.roc_curve(y_test2, grid.predict(X_test2))
print("Ring AUC = %3.3f" %(metrics.auc(fpr, tpr)))
print(grid.best_params_)

### YEAST 1

##### Recursive Feature Elimination (RFE)

In [ ]:
rfe = RFE(grid, 4)
#start = time.process_time()
RFE_X_Train = rfe.fit_transform(X_train3,y_train3)
RFE_X_Test = rfe.transform(X_test3)

rfe = rfe.fit(RFE_X_Train,y_train3)

#print(time.process_time() - start)

print("Overall Accuracy using RFE: ", rfe.score(RFE_X_Test,y_test3))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test3, grid.predict(RFE_X_Test))
print("Elephant AUC = %3.3f" %(metrics.auc(fpr, tpr)))
print(grid.best_params_)

##### Normal

In [ ]:
clf.fit(X_train3,y_train3)
print("Yeast accuracy = %3.3f" %(grid.score(X_test3,y_test3)))
fpr, tpr, thresholds = metrics.roc_curve(y_test3, grid.predict(X_test3))
print("Yeast AUC = %3.3f" %(metrics.auc(fpr, tpr)))
print(grid.best_params_)

## XG Boost

##### Grid Search 

In [102]:
steps = [('scaler', MinMaxScaler()), ('xgbc', XGBClassifier())]
pipeline = Pipeline(steps)

In [104]:
parameters = {'xgbc__n_estimators': range(60, 220, 40),
'xgbc__max_depth':[3, 4, 5],
}

#'xgbc__gamma':[0.5, 1, 1.5, 2, 5],
#'xgbc__learning_rate': [0.1, 0.01, 0.05]

In [105]:
grid_xgbc = GridSearchCV(pipeline, param_grid=parameters, cv=5)

### ELEPHANTS

In [106]:
grid_xgbc.fit(X_train1, y_train1)
print("Elephant accuracy XGBoost = %3.4f" %(grid_xgbc.score(X_test1,y_test1)))
fpr, tpr, thresholds = metrics.roc_curve(y_test1, grid_xgbc.predict(X_test1))
print("Elephant AUC XGBoost = %3.4f" %(metrics.auc(fpr, tpr)))
print(grid_xgbc.best_params_)

Elephant accuracy XGBoost = 0.8208
Elephant AUC XGBoost = 0.8233
{'xgbc__max_depth': 4, 'xgbc__n_estimators': 180}


### RINGS

In [107]:
grid_xgbc.fit(X_train2,y_train2)
print("Ring accuracy XGBoost = %3.4f" %(grid_xgbc.score(X_test2,y_test2)))
fpr, tpr, thresholds = metrics.roc_curve(y_test2, grid_xgbc.predict(X_test2))
print("Ring AUC XGBoost = %3.4f" %(metrics.auc(fpr, tpr)))
print(grid_xgbc.best_params_)

Ring accuracy XGBoost = 0.9635
Ring AUC XGBoost = 0.9633
{'xgbc__max_depth': 3, 'xgbc__n_estimators': 180}


### YEAST 1

In [108]:
grid_xgbc.fit(X_train3,y_train3)
print("Yeast accuracy XGBoost = %3.4f" %(grid_xgbc.score(X_test3,y_test3)))
fpr, tpr, thresholds = metrics.roc_curve(y_test3, grid_xgbc.predict(X_test3))
print("Yeast AUC XGBoost = %3.4f" %(metrics.auc(fpr, tpr)))
print(grid_xgbc.best_params_)

Yeast accuracy XGBoost = 0.8110
Yeast AUC XGBoost = 0.7248
{'xgbc__max_depth': 3, 'xgbc__n_estimators': 60}


## Naive Bayes

In [98]:
nbc = GaussianNB()
nbc.fit(X_train1, y_train1)
print("Elephant accuracy NB = %3.4f" %(nbc.score(X_test1,y_test1)))
fpr, tpr, thresholds = metrics.roc_curve(y_test1, nbc.predict(X_test1))
print("Elephant AUC NB = %3.4f" %(metrics.auc(fpr, tpr)))

Elephant accuracy NB = 0.6953
Elephant AUC NB = 0.7180


In [99]:
nbc = GaussianNB()
nbc.fit(X_train2, y_train2)
print("Ring accuracy NB = %3.4f" %(nbc.score(X_test2,y_test2)))
fpr, tpr, thresholds = metrics.roc_curve(y_test2, nbc.predict(X_test2))
print("Ring AUC NB = %3.4f" %(metrics.auc(fpr, tpr)))

Ring accuracy NB = 0.9811
Ring AUC NB = 0.9811


In [100]:
nbc = GaussianNB()
nbc.fit(X_train3, y_train3)
print("Ring accuracy NB = %3.4f" %(nbc.score(X_test3,y_test3)))
fpr, tpr, thresholds = metrics.roc_curve(y_test3, nbc.predict(X_test3))
print("Ring AUC NB = %3.4f" %(metrics.auc(fpr, tpr)))

Ring accuracy NB = 0.3368
Ring AUC NB = 0.5281


### Bagging

In [141]:
clf1 = DecisionTreeClassifier(criterion='entropy', max_depth=1)
clf2 = KNeighborsClassifier(n_neighbors=1)

In [142]:
steps = [('scaler', StandardScaler()), ('bagging1', BaggingClassifier(base_estimator=clf1))]
steps2 = [('scaler', StandardScaler()), ('bagging2', BaggingClassifier(base_estimator=clf2))]
pipeline = Pipeline(steps)
pipeline2 = Pipeline(steps2)

In [151]:
parameters = {'bagging1__n_estimators': range(5, 50, 5)
}

parameters2 = {'bagging2__n_estimators': range(5, 50, 5)
}

In [152]:
Bagging1 = GridSearchCV(pipeline, param_grid=parameters, cv=5)
Bagging2 = GridSearchCV(pipeline2, param_grid=parameters2, cv=5)

In [153]:
Bagging1.fit(X_train1, y_train1)
print("Elephant accuracy XGBoost = %3.4f" %(Bagging1.score(X_test1,y_test1)))
fpr, tpr, thresholds = metrics.roc_curve(y_test1, Bagging1.predict(X_test1))
print("Elephant AUC XGBoost = %3.4f" %(metrics.auc(fpr, tpr)))
print(Bagging1.best_params_)

Bagging2.fit(X_train1, y_train1)
print("Elephant accuracy XGBoost = %3.4f" %(Bagging2.score(X_test1,y_test1)))
fpr, tpr, thresholds = metrics.roc_curve(y_test1, Bagging2.predict(X_test1))
print("Elephant AUC XGBoost = %3.4f" %(metrics.auc(fpr, tpr)))
print(Bagging2.best_params_)

Elephant accuracy XGBoost = 0.6846
Elephant AUC XGBoost = 0.6956
{'bagging1__n_estimators': 15}
Elephant accuracy XGBoost = 0.7348
Elephant AUC XGBoost = 0.7309
{'bagging2__n_estimators': 10}


In [155]:
Bagging1.fit(X_train2, y_train2)
print("Ring accuracy Bagging = %3.4f" %(Bagging1.score(X_test2,y_test2)))
fpr, tpr, thresholds = metrics.roc_curve(y_test2, Bagging1.predict(X_test2))
print("Ring AUC Bagging = %3.4f" %(metrics.auc(fpr, tpr)))
print(Bagging1.best_params_)

Bagging2.fit(X_train2, y_train2)
print("Ring accuracy Bagging = %3.4f" %(Bagging2.score(X_test2,y_test2)))
fpr, tpr, thresholds = metrics.roc_curve(y_test2, Bagging2.predict(X_test2))
print("Ring AUC Bagging = %3.4f" %(metrics.auc(fpr, tpr)))
print(Bagging2.best_params_)

Ring accuracy Bagging = 0.7257
Ring AUC Bagging = 0.7237
{'bagging1__n_estimators': 5}
Ring accuracy Bagging = 0.7561
Ring AUC Bagging = 0.7538
{'bagging2__n_estimators': 10}


In [156]:
Bagging1.fit(X_train3, y_train3)
print("Ring accuracy Bagging = %3.4f" %(Bagging1.score(X_test3,y_test3)))
fpr, tpr, thresholds = metrics.roc_curve(y_test3, Bagging1.predict(X_test3))
print("Ring AUC Bagging = %3.4f" %(metrics.auc(fpr, tpr)))
print(Bagging1.best_params_)

Bagging2.fit(X_train3, y_train3)
print("Ring accuracy Bagging = %3.4f" %(Bagging2.score(X_test3,y_test3)))
fpr, tpr, thresholds = metrics.roc_curve(y_test3, Bagging2.predict(X_test3))
print("Ring AUC Bagging = %3.4f" %(metrics.auc(fpr, tpr)))
print(Bagging2.best_params_)

Ring accuracy Bagging = 0.7079
Ring AUC Bagging = 0.5000
{'bagging1__n_estimators': 10}
Ring accuracy Bagging = 0.6770
Ring AUC Bagging = 0.5853
{'bagging2__n_estimators': 5}
